In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/github/imdb-sentiment-comparison-rnn-transformer/src/jpynb')

In [3]:
import sys

path = '/content/drive/MyDrive/github/imdb-sentiment-comparison-rnn-transformer/src/python'
sys.path.append(path)


In [11]:
import sklearn
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
import torch
from torch import optim, nn
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# DataLoader

In [4]:
import data

train_loader_rnn = data.train_loader_rnn
test_loader_rnn = data.test_loader_rnn
train_loader_tr = data.train_loader_tr
test_loader_tr = data.test_loader_tr

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#Evaluate def

In [17]:
def evaluation(y, y_proba):
    y_pred = (y_proba > 0.5).astype(int)
    accuracy = accuracy_score(y, y_pred)
    roc = roc_auc_score(y, y_proba)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)

    print(f'Accuracy Score: {np.round(accuracy, 4)}')
    print(f"ROC AUC Score : {np.round(roc, 4)}")
    print(f"Precision      : {np.round(precision, 4)}")
    print(f"Recall         : {np.round(recall, 4)}")

def get_predictions(model, loader, device, is_rnn=True):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        if is_rnn:
            for X_te, lengths, y_te in loader:
                X_te = X_te.to(device)
                y_te = y_te.view(-1)

                logits = model(X_te, lengths)
                probs = torch.sigmoid(logits).view(-1)

                all_probs.append(probs.cpu())
                all_labels.append(y_te.cpu())
        else:
            for X_te, mask, y_te in loader:
                X_te, mask = X_te.to(device), mask.to(device)
                y_te = y_te.view(-1)

                logits = model(X_te, mask)
                probs = torch.sigmoid(logits).view(-1)

                all_probs.append(probs.cpu())
                all_labels.append(y_te.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_labels = torch.cat(all_labels).numpy()
    return all_labels, all_probs

# Train RNN Model

In [18]:
from models import IMDBRNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'

vocab_size = 20000
embedding_dim = 128
hidden_dim = 256
padding_idx = 1

clf_rnn = IMDBRNN(input_size=vocab_size,
                  embed_dim=embedding_dim,
                  hidden_size=hidden_dim,
                  padding_idx=padding_idx).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(clf_rnn.parameters(), lr=0.001)

for epoch in range(10):
  total_loss = 0
  clf_rnn.train()
  for X_tr, lengths, y_tr in train_loader_rnn:
    X_tr, lengths, y_tr = X_tr.to(device), lengths, y_tr.to(device)
    loss = criterion(clf_rnn(X_tr, lengths), y_tr)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f"epoch: {epoch + 1}, loss: {total_loss / len(train_loader_rnn)}")

epoch: 1, loss: 0.6811046083564953
epoch: 2, loss: 0.6678086757050146
epoch: 3, loss: 0.6930491566048254
epoch: 4, loss: 0.6784134712975348
epoch: 5, loss: 0.6359216197372397
epoch: 6, loss: 0.5704841686179266
epoch: 7, loss: 0.5478801867541145
epoch: 8, loss: 0.5450753594755822
epoch: 9, loss: 0.46822814250846045
epoch: 10, loss: 0.44513808857754367


#Train LSTM Model

In [20]:
from models import IMDBLSTM

clf_lstm = IMDBLSTM(input_size=vocab_size,
                  embed_dim=embedding_dim,
                  hidden_size=hidden_dim,
                  padding_idx=padding_idx).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(clf_lstm.parameters(), lr=0.001)

for epoch in range(10):
  total_loss = 0
  clf_lstm.train()
  for X_tr, lengths, y_tr in train_loader_rnn:
    X_tr, lengths, y_tr = X_tr.to(device), lengths, y_tr.to(device)
    loss = criterion(clf_lstm(X_tr, lengths), y_tr)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f"epoch: {epoch + 1}, loss: {total_loss / len(train_loader_rnn)}")

epoch: 1, loss: 0.6551407980339606
epoch: 2, loss: 0.6075652218840616
epoch: 3, loss: 0.38012802951476155
epoch: 4, loss: 0.2366973238108713
epoch: 5, loss: 0.15554291227132158
epoch: 6, loss: 0.09420477478381466
epoch: 7, loss: 0.0515813121829382
epoch: 8, loss: 0.03188567727332568
epoch: 9, loss: 0.020617745710415362
epoch: 10, loss: 0.01649185589915427


#Train Tramsformer Model

In [24]:
from models import TransformerClassifier

vocab_size = 20000
d_model = 128
num_heads = 8
num_layers = 1
d_ff = 512
num_classes = 1
max_len = 512

clf_tr = TransformerClassifier(
    vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    num_classes=num_classes,
    max_len=max_len
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(clf_tr.parameters(), lr=0.001)

In [25]:
for epoch in range(10):
    total_loss = 0
    clf_tr.train()

    for X_tr, mask, y_tr in train_loader_tr:
        X_tr, mask = X_tr.to(device), mask.to(device)
        y_tr = y_tr.to(device).view(-1, 1)

        logits = clf_tr(X_tr, mask)
        loss = criterion(logits, y_tr)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"epoch: {epoch + 1}, loss: {total_loss / len(train_loader_tr):.4f}")

epoch: 1, loss: 0.4929
epoch: 2, loss: 0.3199
epoch: 3, loss: 0.2208
epoch: 4, loss: 0.1496
epoch: 5, loss: 0.0937
epoch: 6, loss: 0.0626
epoch: 7, loss: 0.0419
epoch: 8, loss: 0.0302
epoch: 9, loss: 0.0256
epoch: 10, loss: 0.0204


rnn train score

In [19]:
labels, probs = get_predictions(clf_rnn, train_loader_rnn, device, True)
evaluation(labels, probs)

Accuracy Score: 0.8378
ROC AUC Score : 0.9052
Precision      : 0.8526
Recall         : 0.817


LSTM train score

In [21]:
labels, probs = get_predictions(clf_lstm, train_loader_rnn, device, True)
evaluation(labels, probs)

Accuracy Score: 0.998
ROC AUC Score : 0.9999
Precision      : 0.9968
Recall         : 0.9991


Transformer train score

In [26]:
labels, probs = get_predictions(clf_tr, train_loader_tr, device, is_rnn=False)
evaluation(labels, probs)

Accuracy Score: 0.9976
ROC AUC Score : 1.0
Precision      : 0.9977
Recall         : 0.9974


# Test Score

In [28]:
# 1. RNN
print("="*30)
print("RNN Test Set Evaluation")
labels_rnn_test, probs_rnn_test = get_predictions(clf_rnn, test_loader_rnn, device, is_rnn=True)
evaluation(labels_rnn_test, probs_rnn_test)

# 2. LSTM
print("\n" + "="*30)
print("LSTM Test Set Evaluation")
labels_lstm_test, probs_lstm_test = get_predictions(clf_lstm, test_loader_rnn, device, is_rnn=True)
evaluation(labels_lstm_test, probs_lstm_test)

# 3. Transformer
print("\n" + "="*30)
print("Transformer Test Set Evaluation")
labels_tr_test, probs_tr_test = get_predictions(clf_tr, test_loader_tr, device, is_rnn=False)
evaluation(labels_tr_test, probs_tr_test)
print("="*30)

RNN Test Set Evaluation
Accuracy Score: 0.7123
ROC AUC Score : 0.7745
Precision      : 0.7258
Recall         : 0.6825

LSTM Test Set Evaluation
Accuracy Score: 0.863
ROC AUC Score : 0.9296
Precision      : 0.8538
Recall         : 0.876

Transformer Test Set Evaluation
Accuracy Score: 0.8386
ROC AUC Score : 0.9128
Precision      : 0.8485
Recall         : 0.8244
